In [ ]:
import random, time, cv2
from matplotlib import pyplot as plt
from pydnet import darknet

input_file = './dog.jpg'

config_file = './yolov4-tiny.cfg'
data_file = './coco.data'
weights_file = './yolov4-tiny.weights'

thresh = 0.5

In [ ]:
!wget -nc https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights

In [ ]:
network, class_names, class_colors = darknet.load_network(    
    config_file,
    data_file,
    weights_file,
)    
  
width = darknet.network_width(network)    
height = darknet.network_height(network)    
darknet_image = darknet.make_image(width, height, 3)    

image =  cv2.imread(input_file)    
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
image_resized = cv2.resize(image_rgb, (width, height),    
                           interpolation=cv2.INTER_LINEAR)    

darknet.copy_image_from_bytes(darknet_image, image_resized.tobytes())    
start = time.time()

detections = darknet.detect_image(network, class_names, darknet_image, thresh=thresh)
stop = time.time()


darknet.free_image(darknet_image)    
boxed = darknet.draw_boxes(detections, image_resized, class_colors)
image_boxes = cv2.cvtColor(boxed, cv2.COLOR_BGR2RGB)

darknet.print_detections(detections, True)
print('FPS: {}'.format(int(1/(stop-start))))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,15))
axes[0].imshow(image)
axes[1].imshow(image_boxes)
plt.show()